# Web Based Data Project Python Script: 2-Step Validation Model

### Importing Libraries

In [ ]:
from tqdm import tqdm
import time
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
import simple_icd_10 as icd
import pandas as pd
import csv

In [ ]:
from Bio import Entrez
from Bio.Entrez import efetch, read
Entrez.email = "arthur.hughes27@outlook.com" 

In [ ]:
from pymed import PubMed
pubmed = PubMed(email="arthur.hughes27@outlook.com")

## Retrieve links from pubmed papers using query

### helper function

`mesh_helper(pmid)`: the function `mesh_helper` takes in a PubMed ID and retrieve its MeSH terms if any

`get_link_from_abstract(abstract_lst)`: the function `get_link_from_abstract` takes in a list of abstract and it returns a list of sentences which contaning the target words in each abstract.

`mesh_and_link(pmid_lst, link_lst)`: the function `mesh_and_link` takes in a list of PubMed ID and a list of sentences retrieved from the abstract; returns a dictionary containing the PubMed ID, MeSH terms and a list of sentences retrieved from the abstract.

In [ ]:
def mesh_helper(pmid):
    # credit: https://stackoverflow.com/questions/13652230/cant-get-entrez-to-return-mesh-terms-using-biopython
    # call PubMed API
    handle = efetch(db='pubmed', id=str(pmid), retmode='xml')
    xml_data = read(handle)['PubmedArticle'][0]

    # skip articles without MeSH terms
    if u'MeshHeadingList' in xml_data['MedlineCitation']:
        for mesh in xml_data['MedlineCitation'][u'MeshHeadingList']:
            # grab descriptor name
            name = mesh['DescriptorName'].title()
            descr = [mesh['DescriptorName']]
            # grab descriptor id
            mesh_id = list(descr[0].attributes.items())[0][1]
            major = list(descr[0].attributes.items())[1][1]

            yield(name, mesh_id, major)

def get_link_from_abstract(abstract_lst):
    res_lst = []
    for abstract in abstract_lst:
        # 1. lowercase everything
        text = abstract.lower()
        stemmer = PorterStemmer()
        lines = text.split("\n")
        # store sentence
        res = []
        for l in lines:
            orig_l = l
            # 2. Removing punctuation
            l = l.translate(str.maketrans('', '', string.punctuation))
            # 3. stemming
            l = [stemmer.stem(x) for x in l.split()]   
            l =' '.join(l)
            # target stemmed word
            yes = r"(associ|relat|caus|lead|increas|decreas|result|show|link|affect)"
            # stopwords
            no = r"(investig|can|object|now|recent|whether|worldwid)"
            if re.findall(yes, l) != [] and re.findall(no, l) == []:
                res.append(orig_l)
        res_lst.append(res)
    return res_lst
            
    
def mesh_and_link(pmid_lst, link_lst):
    dlst = {}
    l = 0
    for pmid in tqdm(pmid_lst):
        time.sleep(0.3)
        dlst_key = pmid[0:8]
        helper = mesh_helper(pmid)
        dlst_val = {}
        for name, mesh_id, major in helper:
            # grab mesh term with majortopic = Y only
            if major == "Y":
                dlst_val[mesh_id] = name
        dlst_val['link'] = link_lst[l]
        dlst[dlst_key]=dlst_val
        l += 1
                
    return dlst

### main function

`query_to_link(query, max_num)`: the function `query_to_link` takes in a query and a positive interger. The query is used for PubMining and the `max_num` controls the maximum number for output of the paper retrieved. It returns a dictionary which has PubMed ID as its keys. In the value of each key, it returns a dictionary of MeSH terms and a list of sentences of interest retrieved from the abstract for each paper.

In [ ]:
def query_to_link(query, max_num):
    try:
        results= pubmed.query(query, max_results=max_num)    
        articleList= []
        for article in results:
          articleDict = article.toDict()
          articleList.append(articleDict)

        df= pd.DataFrame(articleList) 
        pmid_lst = df.pubmed_id
        link_lst = get_link_from_abstract(df.abstract)
        
        return mesh_and_link(pmid_lst, link_lst)
    except:
        print("No papers were found. Please modify your query.")

In [ ]:
## example
query_to_link("cardiovascular disease, air pollution[TITLE]", 10)

# 2-Step Validation Model

## Step 1: Validation by CTD

### helper function

`pollutant_ctd_disease(pollutant, num)`: the function `pollutant_ctd_disease` takes two values. `pollutant` is the name of pollutant and it is defined to be one of (O3, PM0.1, PM2.5, PM10, NO, NO2, SO2, CO, NOx). `num` is the number of chemical-disease associations to be retrieved from CTD. It returns a table with three columns. The first column is disease name, the second column is the MeSH ID for the disease, and the third column is the inference score of this association.

`map_ctd(ctd_disease_lst, pred_disease)`: the function `map_ctd` takes in a list of disease name retrieved from CTD and a  disease name from the prediction. It returns a list of disease name retrieved from CTD which are similar to the disease name from the prediction.

In [ ]:
def pollutant_ctd_disease(pollutant, num):
    try:
        if pollutant == "O3":
            pollutant_mesh_id = "D010126"
        if pollutant.startswith("PM"):
            pollutant_mesh_id = "D052638"
        if pollutant == "NO2":
            pollutant_mesh_id = "D009585"
        if pollutant == "NO":
            pollutant_mesh_id = "D009569"
        if pollutant == "SO2":
            pollutant_mesh_id = "D013458"
        if pollutant == "CO":
            pollutant_mesh_id = "D002248"
        if pollutant == "NOx":
            pollutant_mesh_id = "D009589"


        base = "http://ctdbase.org/detail.go?acc="
        # we look into cardiovascular disease only; inferenece are sorted in the order of inference score
        tail = "&view=disease&slimTerm=Cardiovascular+disease&assnType=all&sort=networkScore&6578706f7274=1&type=chem&dir=asc&d-1332398-e=5"
        url =  base+pollutant_mesh_id+tail
        res = pd.read_table(url)[['Disease Name', 'Disease ID', 'Inference Score']]
        if num <= len(res.index):
            return(res.head(num))
        else:
            print("The requested number of output exceeded the maximum. The maximum number of output can be returned is {}.".format(len(res.index)))
        
    except:
        print("Make sure the input is one of (O3, PM0.1, PM2.5, PM10, NO, NO2, SO2, CO, NOx)")

def map_ctd(ctd_disease_lst, pred_disease):
    res = []
    stemmer = PorterStemmer()
    # 1. remove punctuation
    pred_disease = pred_disease.translate(str.maketrans('', '', string.punctuation))
    # 2. stemming
    pred_disease = [stemmer.stem(x) for x in pred_disease.split()]
    
    for d in ctd_disease_lst:
        # 1. lowercase everything
        ctd_d = d.lower()
        # 2. remove punctuation
        ctd_d = ctd_d.translate(str.maketrans('', '', string.punctuation))
        ctd_d = ctd_d.split()
        stop_words = ["diseas"]
        # 3. stemming and remove stopwords
        ctd_d = [stemmer.stem(x) for x in ctd_d if stemmer.stem(x) not in stop_words ]
       
        for t in ctd_d:
            if t in pred_disease:
                res.append(d)
                break
    return res


In [ ]:
## example 1
pollutant_ctd_disease("O3", 10)

In [ ]:
## example 2
map_ctd(pollutant_ctd_disease("O3", 10)['Disease Name'], "Heart Failure")

### main function

`pred_to_ctd(pred_csv, num=10)`: the function `pred_to_ctd` takes in two values. `pred_csv` is the name of csv file stored the prediction (first column: pollutant, second column: ICD 10 code). `num` is the number of chemical-disease associations to be retrieved from CTD (for `pollutant_ctd_disease`). The default `num` is 10.

It returns a dataframe with four columns. The first three columns containing the prediction (pollutant, ICD 10 code of the disease, the name of the disease). The fourth columns containing the matched diseases from CTD. It will either be a list of disease name or "no match".

In [ ]:
def pred_to_ctd(pred_csv, num=10):
    pred = open(pred_csv, 'r')
    pred = csv.reader(pred, delimiter = '\t')
    res = []
    for row in pred:
        time.sleep(0.3)
        pollutant = row[0]
        icd_code = row[1]
        disease = icd.get_description(icd_code).lower()
        
        ctd = pollutant_ctd_disease(pollutant, num)
        
        ctd_res = map_ctd(ctd['Disease Name'], disease)
        
        if ctd_res != []:
            res.append([pollutant, icd_code, disease, ";".join(ctd_res)])
        else:
            res.append([pollutant, icd_code, disease, "no match"])

    res = pd.DataFrame(res, columns=["Pollutant", "ICD 10 Code", "Disease", "CTD"])
    return res

In [ ]:
## res.csv is the prediction file from our study
pred_res = pred_to_ctd('res.csv', 10).head(20)
pred_res

In [ ]:
pred_res.to_csv('pred_validation.csv')

## Step 2: Validation by PubMining

we process the step 2 validation on those links which found no match in the first step

In [ ]:
un_match = pred_res.loc[ pred_res["CTD"] == "no match"]
un_match

### Main function

`pred_to_pub(pollutant, disease, num=10)`: the function `pred_to_pub` takes in a pollutant, a disease and a positive integer. `num` controls the maximum output of PubMining. The default is 10. The function returns a dictionary which containing PubMed ID, MeSH terms and a list of sentences of interest retrieved from the abstract for each paper it mined.

In [ ]:
def pred_to_pub(pollutant, disease, num=10):
    # 1. remove punctuation
    disease = disease.translate(str.maketrans('', '', string.punctuation))
    stopword = stopwords.words("english")
    stopword.extend(["unspecified"])
    # 2. stopwords
    disease = [x for x in disease.split() if x not in stopword]
    disease = ' '.join(disease)
    # 3. build a query to search
    query = pollutant + " " + disease
    return query_to_link(query, num)

In [ ]:
## example: one of the prediction that found no match in step 1 validation
pred_to_pub("SO2", "atherosclerotic heart disease")